In [41]:
import pandas as pd

In [42]:
characteristics_df = pd.read_csv('data/characteristics_prepped.csv', encoding='latin1')
places_df = pd.read_csv('data/places.csv', encoding='latin1')
users_df = pd.read_csv('data/users.csv', encoding='latin1')
vehicles_df = pd.read_csv('data/vehicles_prepped.csv', encoding='latin1')

<ipython-input-42-7febffc0b0d9>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  places_df = pd.read_csv('data/places.csv', encoding='latin1')


## Vehicles + users merge

Vechicles and users table might contain multiple rows for the same Num_Acc id. (Because there might be multiple people or vehicles involved in the accident)

In [43]:
vehicles_df.shape

(1433389, 8)

In [44]:
users_df.shape

(1876005, 12)

In [45]:
vehicles_users_df = pd.merge(vehicles_df, users_df, on='Num_Acc', how='outer', suffixes=('_vehicles', '_users'))

Filter rows that that contains a user-vehicle pair that does not match.

In [46]:
vehicles_users_df = vehicles_users_df[vehicles_users_df['num_veh_vehicles'] == vehicles_users_df['num_veh_users']]

In [47]:
vehicles_users_df.head()

,Num_Acc,Category,Occupants,Fix Obstacle,Mobile Obstacle,Shock,Maneuver,num_veh_vehicles,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh_users
0,201600000001,7,0,0.0,0.0,1.0,1.0,B02,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,B02
3,201600000001,2,0,0.0,0.0,7.0,15.0,A01,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,A01
4,201600000002,7,0,6.0,0.0,1.0,1.0,A01,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0,A01
5,201600000002,7,0,6.0,0.0,1.0,1.0,A01,2.0,2,3,1,0.0,11.0,0.0,0.0,0.0,2000.0,A01
6,201600000002,7,0,6.0,0.0,1.0,1.0,A01,3.0,2,3,2,0.0,11.0,0.0,0.0,0.0,1962.0,A01


In [48]:
vehicles_users_df['num_veh_vehicles'].isna().sum()

0

In [49]:
vehicles_users_df['num_veh_users'].isna().sum()

0

In [50]:
vehicles_users_df.drop(['num_veh_users', 'num_veh_vehicles'], axis=1, inplace=True)

In [51]:
vehicles_users_df.shape

(1875983, 17)

There are no vehicles without users and no users without vehicles.

## Characteristics + places merge

In [52]:
places_df.shape

(839985, 18)

In [53]:
characteristics_df.shape

(839985, 22)

In [54]:
characteristics_places_df = pd.merge(places_df, characteristics_df, on='Num_Acc', how='inner')

In [55]:
characteristics_places_df.shape

(839985, 39)

## Merge all

In [56]:
merged_df = pd.merge(characteristics_places_df, vehicles_users_df, on='Num_Acc', how='inner')

In [57]:
merged_df.shape

(1875983, 55)

## Correlation analysis